## Implementation of Stack, Queues, and Deques

## (1) Stack
#### The list class already supports adding an element to the end with the append method, and removing the last element with the pop method

| Stack Method | Realization with Python List |
|: ----------- |: --------------------------- |
| S.push(e) | L.append(e) |

### LIFO Stack implementation using a Python list as underlying storage.

In [ ]:
class Empty(Exception):
    """Error attempting to access an element from an empty container"""
    pass

class ArrayStack:
    def __init__(self):
        """Create a empty stack"""
        self._data = []        # nonpublic list instance
    
    def __len__(self):
        """Return the # of elements in the stack"""
        return len(self._data)
    
    def is_empty(self):
        """return True if the Stack is empty"""
        return len(self._data) == 0
    
    def push(self, e):
        """Add element e to the top of the stack"""
        self._data.append(e)        # new item stored at the end of list
        
    def top(self):
        """Return (but not remove) the element at the top of the stack"""
        # raise Empty exception if the stack is empty
        if self.is_empty():
            raise Empty('Stack is empty')
        return self._data[-1]        # the last item in the list
    
    def pop(self):
        """Remove and return the element from the top of the stack"""
        # raise Empty exception if the stack is empty
        if self.is_empty():
            raise Empty('Stack is empty')
        return self._data.pop()       # remove the last item from list

### Reversing Data Using a Stack

In [ ]:
def reverse_file(filename):
    """Overwrite given file with its contents line-by-line reversed."""
    S = ArrayStack()
    original = open(filename)
    for line in original:
        S.push(line.rstrip('\n'))         # will re-insert newlines when writing
    original.close()
    
    # now we overwrite with contents with LIFO order
    output = open(filename, 'w')          # reopening file overwrites original
    while not S.is_empty():
        output.write(S.pop() + '\n')      # re-insert newline character
    output.close()

### Matching Delimiters - O(n)

In [ ]:
def is_matched(expr):
    """Return True if all delimiters are properly match; False otherwise"""
    lefty = '({['          # opening delimiters
    righty = ')}]'         # respective closing delims
    S = ArrayStack()
    for c in expr:
        if c in lefty:
            S.push(c)
        elif c in righty:
            if S.is_empty():            # nothing to match with
                return False    
            if righty.index(c) != lefty.index(S.pop()):
                return False            # mismatched
    return S.is_empty()

In [ ]:
def is_matched_html(raw):
    """Return True if all HTML tags are properly match; False otherwise."""
    S = ArrayStack()
    j = raw.find('<')               # find first '<' if any
    while j != -1:
        k = raw.find('>',j+1)       # find next '>' character
        if k == -1:
            return False
        tag = raw[j+1:k]            # strip away < >
        if not tag.startswith('/'):         # this is opening tag
            S.puch(tag)
        else:                               # this is closing tag
            if S.is_empty():
                return False                
            if tag[1:] != S.pop():          # mismatched delimiter
                return False
        j = raw.find('<',k+1)
    return S.is_empty()

## (2) Queues 
#### FIFO queue implementation using a Python List in circular fashion

In [ ]:
class ArrayQueue:
    DEFAULT_CAPACITY = 10       # moderate capacity for all new queues
    
    def __init__(self):
        """Create an empty queue"""
        self._data = [None] * ArrayQueue.DEFAULT_CAPACITY
        self._size = 0
        self._front = 0
        
    def __len__(self):
        """Return the number of elements in the queue"""
        return self._size
    
    def is_empty(self):
        """Retrun True if the queue is empty"""
        return self._size == 0
    
    def first(self):
        """Return (but not remove) the element at the front of the queue.
           Raise Empty exception if the queue is empty.
        """
        if self.is_empty():
            raise Empty('Queue is empty')
        return self._data[self._front]
    
    def dequeue(self):
        """Remove and return the first element of the queue(i.e., FIFO).
           Raise Empty exception if the queue is empty.
        """
        if self.is_empty():
            raise Empty('Queue is empty')
        answer = self._data[self._front]
        self._data[self._front] = None              # help garbage collection
        self._front = (self._front + 1) % len(self._data)
        self._size -= 1
        """Reduce the array to half of its current size whenever the # of elements
           stored in it falls below 1/4 of its capacity
        """
        if 0 < self._size < len(self._data)//4 :
            self._resize(len(self._data) // 2)
        return answer
    
    def enqueue(self, e):
        """Add an element to the back of the queue"""
        if self._size == len(self._data):
            self._resize(2 * len(self._data))       # double the array size
        avail = (self._front + self._size) % len(self._data)        # proper index to place new element
        self._data[avail] = e
        self._size += 1
        
    def _resize(self, cap):     # we assume cap >= len(self)
        """Resize to a new list of capacity >= len(self)."""
        old = self._data        # keep track of existing list
        self._data = [None] * cap       # allocate list with new capacity
        walk = self._front
        for k in range(self._size):         # only consider existing elements
            self._data[k] = old[walk]       # intentionally shift indices
            walk = (1 + walk) % len(old)    # use old size as modulus
        self._front = 0                     # front has been realigned

## (3) Deque (double-ended queue, pronounced "deck")
#### Implementing a Deque with a circular array